In [37]:
import requests
# from lxml import html

from bs4 import BeautifulSoup
import json
import random
import pandas as pd
import plotly.express as px


DownloadNewData = True

if DownloadNewData:
    # session = requests.Session()

    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0",
        "Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0"
    ]
    random_user_agent = random.choice(user_agents)

    headers = {'User-Agent': random_user_agent,
                # 'cookie':'CONSENT=PENDING+660',
                }
    # session.headers.update(headers)
    
    ads_returned = True
    pagenr = 0
    # maxpages = 5
    df_ads = pd.DataFrame()
    while (ads_returned):
        pagenr = pagenr+1
        url = f'https://www.marktplaats.nl/l/caravans-en-kamperen/vouwwagens/p/{pagenr}/#q:holtkamper|PriceCentsTo:1000000|searchInTitleAndDescription:true'
        page = requests.get(url,headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')
        # print(soup.prettify())
        data = json.loads(soup.find('script',type='application/json').text)
        ads = data['props']['pageProps']['searchRequestAndResponse']['listings']
        if len(ads) > 0:
            df_temp = pd.DataFrame(ads)
            df_ads = pd.concat([df_ads,df_temp],ignore_index=True)
        else:
            ads_returned = False
    timestamp = pd.Timestamp.strftime(pd.Timestamp.now(),'%Y%m%dT%H%M')
    df_ads.to_pickle(f'{timestamp}_Marktplaats_vouwwagen_ads.pkl')
else:
    df_ads = pd.read_pickle('Download vouwwagens 20230821.pkl')

# df_ads

1

In [26]:
### drop adevrtisement with "a" (commercial?)
df_ads.drop(df_ads[df_ads['itemId'].str.contains('a')].index,inplace=True)
# df_ads

In [27]:
def get_production_year(x):
    # bla bla
    if pd.isna(x):
        return pd.NA
    else:
        return int(x[0]['value'])
    
def get_price(x):
    # return 1
    # print(x)
    if pd.isna(x):
        return pd.NA
    else:
        return x['priceCents']/100
    
def get_manufacturer(x):
    ManufacturerList = ['Jamet', 'Holtkamper', 'Combi Camp','Aart Kok','Alpenkreuzer','Kip','Cabanon','Campooz','Trigano','Raclet','Camp-Let','Paradiso','Roadmaster','Gerjak','Rapido','3Dog','Conway']
    for Manufacturer in ManufacturerList:
        if x.lower().find(Manufacturer.lower()) > -1:
            return Manufacturer
    return 'Not found in list'


df_ads['price_eur'] = df_ads['priceInfo'].apply(get_price)
df_ads['construction_year'] = df_ads['attributes'].apply(get_production_year)
df_ads['Manufacturer'] = df_ads['title'].apply(get_manufacturer)

df_ads[df_ads['Manufacturer']=='Not found in list'][['itemId','title','description','Manufacturer','construction_year','price_eur']]


,itemId,title,description,Manufacturer,construction_year,price_eur
8,m2007138116,Vouwwagen 2016,Vouwwagen teun ttt4. Tentdoek gemaakt door dor...,Not found in list,2016,4750.0
16,m2009966572,vouwwagen voor de snelle beslisser en vakantie...,Vouwwagen trigano odyssee. Vakantieklaar! 1-2-...,Not found in list,2006,1900.0
18,m2011714862,"VOUWWAGENS komen uit verhuur 1,5 jaar oud 6-7...","Vouwwagens komen uit verhuur 1,5 jaar oud 6-8 ...",Not found in list,2022,5825.0
25,m2011609794,CAMPLET Concorde,Deze fijne snel op te zetten (1 2 3 systeem) v...,Not found in list,1997,350.0
30,m2013636293,Camplet concorde 2004,Camp-let concorde vouwwagen! Bouwjaar:2004 led...,Not found in list,2004,1750.0
...,...,...,...,...,...,...
984,m2003891505,Retro Vouwwagen,Super leuke retro alpenkreutzer vouwwagen incl...,Not found in list,<NA>,0.0
985,m1930976570,Scout vouwwagen 6 persoon inruil auto/brommer ...,Hierbij bied ik te koop leuke nette vouwwagen ...,Not found in list,1996,850.0
987,m2003700586,Coleman Fleetwood Pop-up Vouwwagen Uniek!,Ontdek de ultieme kampeerervaring met onze pra...,Not found in list,<NA>,8500.0
999,m2003489320,Tag Mountain Ranger 2000,Nette taga mountain ranger 2000. Kan zeker nog...,Not found in list,<NA>,699.0


In [28]:
fig = px.scatter(df_ads, x="construction_year", y="price_eur",
                color="Manufacturer",
                #  size='petal_length', 
                hover_data=['itemId','title']
                 )
fig.show()